# **CSC 464 2.0 Computational Biology** 
## **Assignment 1**
## **AS2016332 - P. M. N. Dabare**

## ***Part I - Needleman-Wunsch***

In [0]:
# This method will be used to initialize the scoring scheme. 
def scoring(match=1, mismatch=-1, gap=-1):
  penalty = {'MATCH': match, 'MISMATCH': mismatch, 'GAP': gap}  #A dictionary for all the penalty values.
  return penalty

In [0]:
# Perform alignment according to the Needleman-Wunsch algorithm for Global Alignment
def needle(seq1, seq2):
  m = len(seq1)+1 # Initiation Matrix Size (Rows)
  n = len(seq2)+1 # Initiation Matrix Size (Columns)
  global scoreMatrix 
  scoreMatrix = [[[[None] for i in range(2)] for i in range(n)] for i in range(m)]  # Initiating Score Matrix

  # Matrix Evaluation -----
  # Filling the first column with gap penalty
  for i in range(m):
    scoreMatrix[i][0] = [penalty['GAP']*i,[]]
    
  # Filling the first row with gap penalty
  for j in range(n):
     scoreMatrix[0][j] = [penalty['GAP']*j,[]]
  for i in range(1,m):
    for j in range(1,n):
      score = penalty['MATCH'] if (seq1[i-1] == seq2[j-1]) else penalty['MISMATCH']

      # Getting the diagonal, horizontal and vertical values according to the penalty
      h_val = scoreMatrix[i][j-1][0] + penalty['GAP']
      d_val = scoreMatrix[i-1][j-1][0] + score
      v_val = scoreMatrix[i-1][j][0] + penalty['GAP']
      o_val = [h_val, d_val, v_val]
      scoreMatrix[i][j] = [max(o_val), [i+1 for i,v in enumerate(o_val) if v==max(o_val)]] # horizontal = 1, diagonal = 2, vertical = 3

  overallScore = scoreMatrix[i][j][0]
  score = overallScore
  l_i = i
  l_j = j
  alignments = []
  findPaths(i,j)

  # Compiling alignments based on discovered matrix pathways
  for element in ALIGNMENT_PATHWAYS:
    i = l_i-1
    j = l_j-1
    sideAlignment = ''
    topAlignment = ''
    for n_direction_c in range(len(element)):
        n_direction = element[n_direction_c]
        score = scoreMatrix[i+1][j+1][0]
        if n_direction == '2':
            sideAlignment = sideAlignment + seq1[i]
            topAlignment = topAlignment + seq2[j]
            i=i-1
            j=j-1
        elif n_direction == '1':
            sideAlignment = sideAlignment + "-" # Representing gap using "-"
            topAlignment = topAlignment + seq2[j]
            j=j-1
        elif n_direction == '3':
            sideAlignment = sideAlignment + seq1[i]
            topAlignment = topAlignment + "-" # Representing gap using "-"
            i=i-1
    alignments.append([topAlignment[::-1],sideAlignment[::-1]])

  printAlignments(alignments) # Function to print all alignments

In [0]:
# Nested function to discover new alignment pathways
ALIGNMENT_PATHWAYS = [] # Initiating list of discovered alignment pathways
def findPaths(c_i, c_j, path=''):
  global ALIGNMENT_PATHWAYS
  i = c_i 
  j = c_j 
  if i == 0 and j==0: 
    ALIGNMENT_PATHWAYS.append(path) 
    return 2 
  direction_t = len(scoreMatrix[i][j][1]) 

  while direction_t<=1: 
    n_direction = scoreMatrix[i][j][1][0] if (i != 0 and j != 0) else (1 if i == 0 else (3 if j==0 else 0)) 
    path = path + str(n_direction) 
    if n_direction == 1: 
      j=j-1
    elif n_direction == 2:
      i=i-1
      j=j-1
    elif n_direction == 3:
      i=i-1
    direction_t = len(scoreMatrix[i][j][1])
    if i == 0 and j==0:
      ALIGNMENT_PATHWAYS.append(path)
      return 3
  if direction_t>1:
    for direction_c in range(direction_t):
      n_direction = scoreMatrix[i][j][1][direction_c] if (i != 0 and j != 0) else (1 if i == 0 else (3 if j==0 else 0))
      temp_path = path + str(n_direction)
      if n_direction == 1:
        n_i = i
        n_j=j-1
      elif n_direction == 2:
        n_i=i-1
        n_j=j-1
      elif n_direction == 3:
        n_i=i-1
        n_j = j
      findPaths(n_i,n_j,temp_path)
     
  return len(ALIGNMENT_PATHWAYS)

In [0]:
# Function to print alignments
def printAlignments(alignments):
  print("Sequences are "+seq1+" and "+seq2+"\n")
  count=1
  for element in alignments:
    print("Alignment "+str(count) +": "+ element[0]+" and "+element[1]+"\n")
    count=count+1
  return

In [5]:
seq1 = "ATTAC"  # Sequence 1 (Side Sequence)
seq2 = "AATTC"  # Sequence 2 (Top Sequence) 
penalty = scoring() # Default penalty values are used
needle(seq1, seq2)

Sequences are ATTAC and AATTC

Alignment 1: AATT-C and A-TTAC

Alignment 2: AATT-C and -ATTAC



## ***Part II - Smith-Waterman***

In [0]:
def waterman(seq1, seq2):
  # Initialize variables and matrices
  cols = len(seq1)
  rows = len(seq2)
  matrix = [[0 for row in range(rows+1)] for col in range(cols+1)]
  paths = [[0 for row in range(rows+1)] for col in range(cols+1)]
  max_score = 0

  # Filling the scoring matrix
  for i in range(cols):
    for j in range(rows):
      if seq1[i] == seq2[j]:
        diag = matrix[i][j] + penalty['MATCH']
      else:
        diag = matrix[i][j] + penalty['MISMATCH'] 
      up    = matrix[i + 1][j] + penalty['GAP']
      left  = matrix[i][j + 1] + penalty['GAP']
      score = max(0,diag, up, left)    
      matrix[i+1][j+1] = score  
      if score > max_score:
        max_score = score
        
      # Filling the paths matrix
      if matrix[i+1][j+1]   == diag and matrix[i+1][j+1] != 0:
        paths[i+1][j+1] = 'diag'
      elif matrix[i+1][j+1] == up   and matrix[i+1][j+1] != 0:
        paths[i+1][j+1] = 'up'
      elif matrix[i+1][j+1] == left and matrix[i+1][j+1] != 0:
        paths[i+1][j+1] = 'left'

  #Traceback   
  maximum=[]  #Starting positions for traceback
  for a in range(cols+1):
    for b in range(rows+1):
      if matrix[a][b]==max_score:
        maximum.append([a,b])
  alignments = []
  for a in maximum:
    i,j=a
    start_path = paths[i][j]
    s1 = '' 
    s2 = ''
    while start_path != 0:
      if start_path == 'diag':
        s1 = s1 + seq1[i-1]
        s2 = s2 + seq2[j-1]
        i, j = i-1, j-1    
      elif start_path == 'up':
        s1 = s1 + '-'
        s2 = s2 + seq2[j-1]
        j = j-1
      else:
        s1 = s1 + seq1[i-1]
        s2  = s2 + '-'
        i = i-1
      start_path = paths[i][j]
    alignments.append([s1[::-1],s2[::-1]])
    
  printAlignments(alignments) # Print the result

In [0]:
# Print the result
def printAlignments(alignments):
  print("Sequences are "+seq1+" and "+seq2+"\n")
  count=1
  for element in alignments:
    print("Alignment "+str(count) +": "+ element[0]+" and "+element[1]+"\n")
    count=count+1
  return

In [8]:
# Input sequences
seq1='ACATAG'
seq2='AATG'
 
penalty=scoring() # Scoring function
waterman(seq1,seq2)

Sequences are ACATAG and AATG

Alignment 1: AT and AT

Alignment 2: ATAG and AT-G

